<a href="https://colab.research.google.com/github/amyjchao/data-course-sample/blob/main/ACHW2_Content_based_RecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# 此內容會顯示為程式碼
```

# HW2-實作 Content-based Filtering 推薦算法

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-02 14:02:15--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.33’

All_Beauty.csv.33   100%[===================>]  14.78M  19.1MB/s    in 0.8s    

2022-01-02 14:02:16 (19.1 MB/s) - ‘All_Beauty.csv.33’ saved [15499476/15499476]

--2022-01-02 14:02:16--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.33’

meta_All_Beauty.jso 100%[===================>]   9.85M  15.2MB/s    in 0.6s

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [5]:
#ratings.head()

## 資料整理

In [6]:
import string

In [7]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [8]:
metadata = metadata[['asin', 'brand', 'title','rank','description']]
metadat = metadata.drop_duplicates('title')
#metadata.head()

In [9]:
metadata['category'] = metadata['rank'].str.split(' in ').str[1]
metadata['rank'] = metadata['rank'].str.split(expand = True)[0]
metadata['category'] = metadata['category'].str[:-2]
metadata['category'].unique()

array(['Beauty & Personal Care', 'Beauty &amp; Personal Care', nan,
       'Grocery & Gourmet Food', 'Grocery &amp; Gourmet Food',
       'Health &amp; Household', 'Sports & Outdoors',
       'Health & Household', 'Toys & Games', 'Baby',
       'Clothing, Shoes & Jewelry', 'Home & Kitchen', 'Automotive',
       'Tools & Home Improvement'], dtype=object)

In [10]:
metadata['rank'] = metadata['rank'].str.replace(',','')
metadata['rank'] = pd.to_numeric(metadata['rank'])

In [ ]:
metadata['description'] = metadata['description'].apply(lambda x: ' '.join(x))
metadata['desc'] = metadata["category"] +' '+metadata["brand"] +' '+ metadata["title"] +' '+ metadata["description"]

In [12]:
metadata['desc'] = metadata['desc'].str.lower()
metadata['desc'] = metadata['desc'].str.translate(str.maketrans('', '', string.punctuation))

In [13]:
metadata = metadata[['asin','title','rank','desc']]

In [14]:
metadata.head()

,asin,title,rank,desc
0,6546546450,Loud 'N Clear&trade; Personal Sound Amplifier,2938573.0,beauty personal care idea village loud n clea...
1,7178680776,No7 Lift &amp; Luminate Triple Action Serum 50...,872854.0,beauty personal care no7 lift amp luminate t...
2,7250468162,No7 Stay Perfect Foundation Cool Vanilla by No7,956696.0,beauty personal care no7 no7 stay perfect fou...
3,7367905066,Wella Koleston Perfect Hair Colour 44/44 Mediu...,1870258.0,beauty personal care wella koleston perfect ...
4,7414204790,Lacto Calamine Skin Balance Oil control 120 ml...,67701.0,beauty personal care pirmal healthcare lacto ...


## 資料切分

In [15]:
#上次發現8周～52周的 & rating >4 的recall是最高的
ratings_trainings = ratings[
    (ratings['DATE'] >= '2018-07-01') & (ratings['DATE'] < '2018-09-01') & (ratings['overall'] >= 4)
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 產生推薦

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
# 每個商品回傳 k 個最相近的商品
def title_recommend_item(item_input, k=2,method='title'):
    try:
        if method == 'title':
          # 計算商品用標題所表示的 tfidf 矩陣
          title_df = metadata.drop_duplicates('title')
          df = title_df
          title_tf = TfidfVectorizer(analyzer='word', stop_words='english')
          title_tfidf_matrix = title_tf.fit_transform(title_df['title'])

          # 計算商品間的相似程度
          title_similarity_matrix = cosine_similarity(title_tfidf_matrix)
          t_mapping = pd.Series(title_df.index,index = title_df['title'])
          item_index = t_mapping[item_input]
          similarity_score = list(enumerate(title_similarity_matrix[item_index]))
        elif method == 'desc':
          # 計算用描述所表示的 tfidf 矩陣
          desc_df = metadata.drop_duplicates('desc')
          df = desc_df
          desc_tf = TfidfVectorizer(stop_words='english')
          desc_tfidf_matrix = desc_tf.fit_transform(desc_df['desc'])

          desc_similarity_matrix = cosine_similarity(desc_tfidf_matrix)
          d_mapping = pd.Series(desc_df.index,index = desc_df['desc'])
          item_index = d_mapping[item_input]
          similarity_score = list(enumerate(desc_similarity_matrix[item_index]))
        else:
          similarity_score = []
        
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

# 利用使用者購買過的商品產生推薦
def recommend_items(items, k, m):
    res = []
    for d in items:
        res.extend(title_recommend_item(d, k, m))
    return res

In [18]:
product_rank = metadata[['asin','rank']]
combine = ratings_trainings.set_index('asin').join(product_rank.set_index('asin'))
mean_df = combine.groupby('asin').mean().reset_index()

In [ ]:
def recommender(training_data, users=[], k=10, method='title'):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    ratings_trainings = training_data
    df = mean_df[mean_df.overall >= 4].sort_values('rank').head(k) 
    for user in users:
      rec = []

      if method == 'title':
        list_of_ratings =metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist()
      elif method == 'desc':
        list_of_ratings =metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['desc'].tolist()
      else:
        list_of_ratings=[]

      if len(list_of_ratings) > 0:
        rec.extend(recommend_items(list_of_ratings,k,method))
      rec.extend(df['asin'])
      rec = rec[:k]
      recommendations[user] = rec
    return recommendations

ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

## 結果評估

In [20]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.020338983050847456

# 數據比較

In [21]:
import numpy as np

In [22]:
ks = range(10,51,10)
ms =['title','desc']
scores = []
for k in ks:
  for m in ms:  
    ratings_by_user = recommender(ratings_trainings, users, k=k,method=m)
    score = evaluate(ratings_testings_by_user, ratings_by_user)
    scores.append(score)
    print("k = %d, method = %s, score = %.5f" %(k, m, score))


pd.DataFrame(np.reshape(scores, (5, 2)).T, index=pd.Index(ms, name='methods'), columns=pd.Index(ks, name='k'))

k = 10, method = title, score = 0.02034
k = 10, method = desc, score = 0.02034
k = 20, method = title, score = 0.04237
k = 20, method = desc, score = 0.04237
k = 30, method = title, score = 0.05424
k = 30, method = desc, score = 0.05424
k = 40, method = title, score = 0.08983
k = 40, method = desc, score = 0.08983
k = 50, method = title, score = 0.10339
k = 50, method = desc, score = 0.10339


k,10,20,30,40,50
methods,,,,,
title,0.020339,0.042373,0.054237,0.089831,0.10339
desc,0.020339,0.042373,0.054237,0.089831,0.10339
